### check installation is success or not
### チュートリアルで利用するパッケージをインポートしてみましょう。エラーが出なければOKです。

In [6]:
from maize.core.workflow import Workflow
from maize.utilities.chem import IsomerCollection
from maize.steps.mai.molecule import Gypsum
from maize.steps.mai.molecule import Smiles2Molecules, LoadSmiles, LoadMolecule
from maize.steps.mai.docking import Vina, VinaScore
from maize.steps.io import LoadData, LoadFile, Return, Void
from pathlib import Path

In [7]:
from rdkit import Chem
import scipy
import meeko

In [8]:
from maize import maize
import rdkit


In [9]:
print(maize.__version__)
print(rdkit.__version__)
print(meeko.__version__)

0.8.3
2024.03.5
0.5.0


#### maizeの設定に関して
- maizeはmaize.tomlというファイルに各種の設定を追記することで外部プログラムをコールしたりすることができます。
- デフォルトではXDG_CONFIG_HOME以下にあるmaize.tomlを見に行くように設定されています。
- XDG_CONFIG_HOMEが環境変数として設定されておらず、maize.tomlが見に行けない場合エラーが起きるかもしれません。
- MacOSは同じ機能を担うディレクトリがあるもののXDG_CONFIG_HOMEは設定されていないかもしれません。
- https://stackoverflow.com/questions/3373948/equivalents-of-xdg-config-home-and-xdg-data-home-on-mac-os-x
- 今回利用する設定ファイルは以下のファイルです。
- Pathは各自で異なると思いますので適宜書き換えてください。

```bash
#just example
!cat../data/maize.toml
[vina]
python = "/home/iwatobipen/miniforge3/envs/maize-dev/bin/python"
commands.vina = "/opt/vina/bin/vina"

[gypsum]
scripts.gypsum.interpreter = "/home/iwatobipen/miniforge3/envs/gypsum/bin/python"
scripts.gypsum.location = "/home/iwatobipen/dev/gypsum_dl/run_gypsum_dl.py"
```
- これを設定ファイルとして利用する際は以下の記載を追記すればよいです。

```python
from maize.utilities.io import Config, NodeConfig
flow.config = Config()
flow.config.update(Path("path/to/config.toml"))
```
- [参考](https://molecularai.github.io/maize/docs/userguide.html)


In [10]:
from maize.utilities.io import Config, NodeConfig
from maize.core.workflow import Workflow
flow = Workflow()
flow.config = Config()
flow.config.update(Path("../data/maize.toml"))

In [14]:
from pprint import pprint
pprint(flow.config)

Config(packages=['maize.steps.mai', 'maize.graphs.mai'],
       scratch=PosixPath('/tmp/tmpvsi5tv5d'),
       batch_config=ResourceManagerConfig(system='local',
                                          max_jobs=100,
                                          queue=None,
                                          project=None,
                                          launcher=None,
                                          walltime='24:00:00',
                                          polling_interval=120),
       environment={},
       nodes={'gypsum': NodeConfig(python=PosixPath('/home/iwatobipen/miniforge3/envs/maize-dev/bin/python'),
                                   modules=[],
                                   commands={},
                                   scripts={'gypsum': {'interpreter': '/home/iwatobipen/miniforge3/envs/gypsum/bin/python',
                                                       'location': PosixPath('/home/iwatobipen/dev/gypsum_dl/run_gypsum_dl.py')}},
     